In [1]:
#@title Install PyRadiomics and import required libraries
!pip install pyradiomics >/dev/null
import yaml
import os
import pandas as pd

In [2]:
#@title Download data
#@markdown Lets know download the folder containing the example images and mask
#@markdown that we will use for the feature extractionn demonstrationns was well as the radiomic feature extraction parameter file.

!wget -q --max-redirect=20 -O ProstateXSegmentationsExample.zip https://www.dropbox.com/s/qp40gx8cnuff2k6/ProstateXSegmentationsExample.zip?dl=1 > /dev/null
!wget -q --max-redirect=20 -O Params_ProstateX_T2WAx_Gland.yaml https://www.dropbox.com/s/th0eq5dds1p0fnr/Params_ProstateX_T2WAx_Gland.yaml?dl=1 > /dev/null
!unzip ProstateXSegmentationsExample.zip >/dev/null
!rm ProstateXSegmentationsExample.zip >/dev/null
!rm -r __MACOSX >/dev/null
# !rm -r ProstateXSegmentationsExample
print('Done!')

Done!


### Lets have a look at our parameter file

Select the folder icon on left bar and double click on Params_ProstateX_T2WAx_Gland.yaml file to see it.

#### Normalization
Since we are working T2w MRI images we will normalize the images by using the parameter `normalize` which we set as `true`. This set the mean intensity of the images at zero and scale the standard deviation to be equal to one. We then scale the standard deviation of the image intensities to be 100 by setting `normalizeScale` to `true`.

The computation of some features (namely Energy, Total Energy and RMS) is affected by the existance of negative value and since we have normalized our images to have mean 0 and standard deviation of 100 we need to ensure that the majority of our normalized image intensities is equal or above 0. We can do this using the `voxelArrayShift`. We selected a value of `300`, which assuming a normal distribution of the image intensities would ensure that 99.7% of them are above 0.

#### Memory savings
The `preCrop` set to `true` allows us to save some memory during feature computation (and possibly preventing errors).

#### Feature extraction type (2D vs 3D)
We have a dataset where our images are anisotropic (voxel dimensions are not the same for all three axes) and, more specifically, the slice thickness is much higher than the in plane resolution. Because of this, we will perform the extraction of 2D radiomic features instead of 3D. Since our images are T2w Axial images we need to define that we wish to perform a 2D feature extraction by setting `force2D` to `true` and set the in-plane axis to `0` using `force2Ddimension` (Axial - 0, Coronal - 1, Saggital - 2).

#### Interpolation/Resampling
To ensure that we are extracting radiomic feature that are comparable among different images we need to standardize the in-plane voxel dimensions. We do this by setting the `interpolator` to `sitkBSpline` and the `resampledPixelSpacing` to a chosen resolution `[0.6, 0.6, 0]` (the 0 on the third position means that we will not resample the slice thickness and only the in-plane resolution of the images will be transformed to 0.6 mm x 0.6 mm, which is ok because the features will be extracted in 2D).

#### Resegmentation
The resegmentation allows the mask to be redefine to, for example, discard intensities in our mask that are outside $mean\ \pm\ 3\times std$. We can do this by setting `resegmentMode` to `sigma` and `resegmentRange` to `[-3, 3]`.

#### Intensity discritization
To ensure that across images the number of bins used for the feature extraction we are between have across patients 30 to 130 performed a first feature extraction to get the `original_firstorder_Range` of our patients and we select the `binWidth` so that it is higher than 30 and lower than 130 accross all patients.

#### Filters and Features
We use the `imageType` to define the types of images that from which we which to extract the radiomic features and `featureClass` to select the type of features to be extracted.

In [ ]:
#@title Perform Radiomic Feature Extraction - Input File

#@markdown For the radiomic feature extraction we will use PyRadiomics batch
#@markdown mode, where we will have to provide as input `.csv` file contain 3
#@markdown columns. The first column is the `ID`, the second the `Image` and the
#@markdown third the `Mask`, as shown below.

input_df = pd.read_csv('/content/ProstateXSegmentationsExample/input_prostateX_T2WAx.csv')
input_df.head()

#@markdown Besides the input file we will have to provide the filepath for the
#@markdown output file where the feature values for the different images will be
#@markdown saved (use `-o` before indicating the path), define that we want the
#@markdown output as a csv file by adding `-f csv`, and finally indicating the
#@markdown filepath of the parameter file using `-p`

#@markdown Our command looks like this:

#@markdown `pyradiomics /content/ProstateXSegmentationsExample/input_prostateX_T2WAx.csv -o /content/radiomics_prostateX_T2WAx.csv -f csv -p /content/Params_ProstateX_T2WAx_Gland.yaml`

,ID,Image,Mask
0,Prostatex0014,/content/ProstateXSegmentationsExample/Prostat...,/content/ProstateXSegmentationsExample/Prostat...
1,Prostatex0023,/content/ProstateXSegmentationsExample/Prostat...,/content/ProstateXSegmentationsExample/Prostat...
2,Prostatex0029,/content/ProstateXSegmentationsExample/Prostat...,/content/ProstateXSegmentationsExample/Prostat...
3,Prostatex0039,/content/ProstateXSegmentationsExample/Prostat...,/content/ProstateXSegmentationsExample/Prostat...
4,Prostatex0048,/content/ProstateXSegmentationsExample/Prostat...,/content/ProstateXSegmentationsExample/Prostat...


In [ ]:
#@title Perform Radiomic Feature Extraction - Required inputs to PyRadiomics batch execution

#@markdown Besides the input file we will have to provide the filepath for the
#@markdown output file where the feature values for the different images will be
#@markdown saved (use `-o` before indicating the path), define that we want the
#@markdown output as a csv file by adding `-f csv`, and finally indicating the
#@markdown filepath of the parameter file using `-p`

#@markdown Our command looks like this:

#@markdown `pyradiomics /content/ProstateXSegmentationsExample/input_prostateX_T2WAx.csv -o /content/radiomics_prostateX_T2WAx.csv -f csv -p /content/Params_ProstateX_T2WAx_Gland.yaml`

In [ ]:
#@title Perform Radiomic Feature Extraction - Execute Feature Extraction
!pyradiomics /content/ProstateXSegmentationsExample/input_prostateX_T2WAx.csv -o /content/radiomics_prostateX_T2WAx.csv -f csv -p /content/Params_ProstateX_T2WAx_Gland.yaml  >/dev/null


[2021-03-18 10:58:31] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2021-03-18 10:58:31] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2021-03-18 10:58:31] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2021-03-18 10:58:31] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2021-03-18 10:58:31] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2021-03-18 10:58:31] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2021-03-18 10:58:31] W: radiomics.glcm: GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
[2021-03-18 10:58:32] W: radiomics.glcm: GLCM is symmet

In [ ]:
#@title Lets check the radiomics output file

#@markdown As you can see it containts the same three columns (ID, Image, Mask)
#@markdown of our input, tnen collumns providing information about the versions,
#@markdown image properties, and feature extraction settings, followed by the
#@markdown values of the extracted radiommic features.

radiomics_df = pd.read_csv('/content/radiomics_prostateX_T2WAx.csv')
radiomics_df.head()

,ID,Image,Mask,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,diagnostics_Mask-original_Spacing,diagnostics_Mask-original_Size,diagnostics_Mask-original_BoundingBox,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,diagnostics_Mask-original_CenterOfMassIndex,diagnostics_Mask-original_CenterOfMass,diagnostics_Image-interpolated_Spacing,diagnostics_Image-interpolated_Size,diagnostics_Image-interpolated_Mean,diagnostics_Image-interpolated_Minimum,diagnostics_Image-interpolated_Maximum,diagnostics_Mask-interpolated_Spacing,diagnostics_Mask-interpolated_Size,diagnostics_Mask-interpolated_BoundingBox,diagnostics_Mask-interpolated_VoxelNum,diagnostics_Mask-interpolated_VolumeNum,diagnostics_Mask-interpolated_CenterOfMassIndex,diagnostics_Mask-interpolated_CenterOfMass,diagnostics_Mask-interpolated_Mean,diagnostics_Mask-interpolated_Minimum,diagnostics_Mask-interpolated_Maximum,...,exponential_glrlm_RunPercentage,exponential_glrlm_RunVariance,exponential_glrlm_ShortRunEmphasis,exponential_glrlm_ShortRunHighGrayLevelEmphasis,exponential_glrlm_ShortRunLowGrayLevelEmphasis,exponential_glszm_GrayLevelNonUniformity,exponential_glszm_GrayLevelNonUniformityNormalized,exponential_glszm_GrayLevelVariance,exponential_glszm_HighGrayLevelZoneEmphasis,exponential_glszm_LargeAreaEmphasis,exponential_glszm_LargeAreaHighGrayLevelEmphasis,exponential_glszm_LargeAreaLowGrayLevelEmphasis,exponential_glszm_LowGrayLevelZoneEmphasis,exponential_glszm_SizeZoneNonUniformity,exponential_glszm_SizeZoneNonUniformityNormalized,exponential_glszm_SmallAreaEmphasis,exponential_glszm_SmallAreaHighGrayLevelEmphasis,exponential_glszm_SmallAreaLowGrayLevelEmphasis,exponential_glszm_ZoneEntropy,exponential_glszm_ZonePercentage,exponential_glszm_ZoneVariance,exponential_ngtdm_Busyness,exponential_ngtdm_Coarseness,exponential_ngtdm_Complexity,exponential_ngtdm_Contrast,exponential_ngtdm_Strength,exponential_gldm_DependenceEntropy,exponential_gldm_DependenceNonUniformity,exponential_gldm_DependenceNonUniformityNormalized,exponential_gldm_DependenceVariance,exponential_gldm_GrayLevelNonUniformity,exponential_gldm_GrayLevelVariance,exponential_gldm_HighGrayLevelEmphasis,exponential_gldm_LargeDependenceEmphasis,exponential_gldm_LargeDependenceHighGrayLevelEmphasis,exponential_gldm_LargeDependenceLowGrayLevelEmphasis,exponential_gldm_LowGrayLevelEmphasis,exponential_gldm_SmallDependenceEmphasis,exponential_gldm_SmallDependenceHighGrayLevelEmphasis,exponential_gldm_SmallDependenceLowGrayLevelEmphasis
0,Prostatex0014,/content/ProstateXSegmentationsExample/Prostat...,/content/ProstateXSegmentationsExample/Prostat...,v3.0.1,1.19.5,2.0.2,1.1.1,3.7.10,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'LoG': {'sigma': [3, 5]}, 'Wa...",c16babc0c68c81ac92970854b980dfee4a5f136c,3D,"(0.5, 0.5, 3.0000007152557373)","(384, 384, 19)",224.535214,0.0,1065.0,06ef90b85db04b9ecf100467e794c0354986e829,"(0.5, 0.5, 3.0000007152557373)","(384, 384, 19)","(149, 166, 4, 91, 68, 15)",35003,1,"(193.09330628803247, 198.6340885066994, 9.5433...","(-16.62551940263117, 35.68184289389275, 12.771...","(0.6, 0.6, 3.0000007152557373)","(88, 69, 19)",6.478077,-148.098153,329.303403,"(0.6, 0.6, 3.0000007152557373)","(88, 69, 19)","(6, 6, 4, 76, 57, 15)",24334,1,"(42.72158297032958, 33.35341497493219, 9.54154...","(-16.689222764449624, 35.62657559464135, 12.77...",-14.609330,-145.098995,329.303403,...,0.044182,287.613363,0.121375,0.335531,0.067836,53.481013,0.676975,0.161513,3.392405,6.985922e+05,6.985993e+05,6.985904e+05,0.40